## Custom YOLO11 Object Detection Model — Google Colab Version
- Dataset: Downloaded from Roboflow (≥100 images)
- Model: YOLO11n (nano)
- Inference: Batch process 20 test images → output saved to folder
- No manual image-by-image handling — fully automated!

In [1]:
# Install required packages
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.4 MB/s eta 0:00:00


In [2]:
# Import libraries
import os
import cv2

In [5]:
# load downloaded zip file from roboflow and unzip to work here
# file uploaded manually or linked from Drive

zip_file_path = "/content/Basketball Players.v1-1.yolov11.zip"
dataset_path = "/content/Basketball_Players_dataset"

if os.path.exists(zip_file_path):
    print(f"Attempting to unzip '{zip_file_path}' using shell command...")
    # Use the unzip shell command
    unzip_command = f"unzip -o \"{zip_file_path}\" -d \"{dataset_path}\""
    result = os.system(unzip_command)

    if result == 0:
        print(f"Unzipped '{zip_file_path}' successfully.")
    else:
        print(f"Error: Failed to unzip '{zip_file_path}'. The file might be corrupted or not a valid zip file.")
else:
    print(f"Error: File '{zip_file_path}' not found. Please ensure the file is in your Google Drive and the path is correct.")

Attempting to unzip '/content/Basketball Players.v1-1.yolov11.zip' using shell command...
Unzipped '/content/Basketball Players.v1-1.yolov11.zip' successfully.


In [6]:
# STEP 2: TRAIN CUSTOM YOLO11 MODEL

from ultralytics import YOLO

# Load a pre-trained YOLO model (you can choose n, s, m, l, or x versions)
model = YOLO("yolo11n.pt")

# Start training on your custom dataset
model.train(data=f'{dataset_path}/data.yaml', epochs=100, imgsz=640, patience=10)

print("Training completed!")

Ultralytics 8.3.206 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Basketball_Players_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, p

In [7]:
# STEP 3: BATCH INFERENCE ON TEST IMAGES
# Get test images from the dataset's test folder
test_images_folder = f"{dataset_path}/test/images"


In [8]:
# Create output folder
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

In [9]:
# List all test images (jpg, png, jpeg)
test_images = [f for f in os.listdir(test_images_folder)
               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"\nFound {len(test_images)} test images. Running inference...")


Found 24 test images. Running inference...


In [11]:
# Load trained model
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Perform inference on all test images
results = model(test_images_folder)


image 1/24 /content/Basketball_Players_dataset/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-106_jpg.rf.58f791f9075215a56c053df5a4da41e3.jpg: 384x640 1 Hoop, 12 Players, 2 Refs, 78.9ms
image 2/24 /content/Basketball_Players_dataset/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-124_jpg.rf.9ab4d3c4f1655d8d518d67365b76a7d9.jpg: 384x640 1 Hoop, 11 Players, 3 Refs, 15.4ms
image 3/24 /content/Basketball_Players_dataset/test/images/Iowa_Nebraska-2023-03-06-college-men_mp4-97_jpg.rf.22bbfcd6b03d2e0935b76558d9917a75.jpg: 384x640 1 Hoop, 13 Players, 2 Refs, 11.3ms
image 4/24 /content/Basketball_Players_dataset/test/images/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg: 384x640 1 Hoop, 11 Players, 2 Refs, 1 Team Points, 16.0ms
image 5/24 /content/Basketball_Players_dataset/test/images/youtube-114_jpg.rf.ff9b1f0ec33de13a86c7b9f583741d4d.jpg: 384x640 1 Period, 4 Players, 3 Team Pointss, 3 Time Remainings, 14.5ms
image 6/24 /content/Basketball_Players_dataset/test/images/youtub

In [12]:
# Loop through all test images
for i, result in enumerate(results[:20]):  # Process first 20 images and iterate through results
    img_file = test_images[i] # Get the original image filename
    output_path = os.path.join(output_folder, img_file)

    # Save result image with bounding boxes
    result.save(filename=output_path)

    print(f"Saved: {output_path}")

print(f"\nProcessed and saved results for the first 20 test images to '{output_folder}' folder.")

Saved: output/youtube-13_jpg.rf.a777dbe3471edec0664f442edd7dd12e.jpg
Saved: output/youtube-70_jpg.rf.c236f2a572f6ee4adcb3f67cc17d007e.jpg
Saved: output/youtube-2_jpg.rf.4f126ce80ed6e9775f3fcc08d01b2a14.jpg
Saved: output/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg
Saved: output/youtube-128_jpg.rf.030a921e404cae04553ed5c196d180c5.jpg
Saved: output/youtube-23_jpg.rf.0456f7fe54cb39c07776fe9a80a40ae4.jpg
Saved: output/Iowa_Nebraska-2023-03-06-college-men_mp4-97_jpg.rf.22bbfcd6b03d2e0935b76558d9917a75.jpg
Saved: output/youtube-153_jpg.rf.3932aa7247ad95a56420c82dd29f2bcf.jpg
Saved: output/youtube-201_jpg.rf.6461310cae11facb5e80b1456f5eb21e.jpg
Saved: output/youtube-230_jpg.rf.d27e969ce96580f5bac466645343d68a.jpg
Saved: output/youtube-9_jpg.rf.f1960ece0187489ee1d5d6b915f84ab7.jpg
Saved: output/youtube-15_jpg.rf.deab89215f9fb6fcadad7234b46c5d5d.jpg
Saved: output/Iowa_Nebraska-2023-03-06-college-men_mp4-106_jpg.rf.58f791f9075215a56c053df5a4da41e3.jpg
Saved: output/youtube-137_jpg.rf.21

In [13]:
# Download output folder as ZIP
from google.colab import files
!zip -r output.zip output/
files.download('output.zip')

  adding: output/ (stored 0%)
  adding: output/youtube-13_jpg.rf.a777dbe3471edec0664f442edd7dd12e.jpg (deflated 3%)
  adding: output/youtube-70_jpg.rf.c236f2a572f6ee4adcb3f67cc17d007e.jpg (deflated 3%)
  adding: output/youtube-2_jpg.rf.4f126ce80ed6e9775f3fcc08d01b2a14.jpg (deflated 3%)
  adding: output/youtube-0_jpg.rf.b22c9e3340726a8c836515dfc12f14b8.jpg (deflated 3%)
  adding: output/youtube-128_jpg.rf.030a921e404cae04553ed5c196d180c5.jpg (deflated 5%)
  adding: output/youtube-23_jpg.rf.0456f7fe54cb39c07776fe9a80a40ae4.jpg (deflated 3%)
  adding: output/Iowa_Nebraska-2023-03-06-college-men_mp4-97_jpg.rf.22bbfcd6b03d2e0935b76558d9917a75.jpg (deflated 4%)
  adding: output/youtube-153_jpg.rf.3932aa7247ad95a56420c82dd29f2bcf.jpg (deflated 3%)
  adding: output/youtube-201_jpg.rf.6461310cae11facb5e80b1456f5eb21e.jpg (deflated 10%)
  adding: output/youtube-230_jpg.rf.d27e969ce96580f5bac466645343d68a.jpg (deflated 3%)
  adding: output/youtube-9_jpg.rf.f1960ece0187489ee1d5d6b915f84ab7.jpg (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>